In [49]:
import numpy as np
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
    load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()
import tsfresh
from tsfresh.feature_extraction import feature_calculators
from tsfresh.utilities.string_manipulation import get_config_from_string
from tempfile import TemporaryFile

In [8]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 10/10 [00:11<00:00,  1.11s/it]


In [9]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)
features_filtered = select_features(extracted_features, y)

/home/thiago/anaconda3/lib/python3.7/site-packages/tsfresh/utilities/dataframe_functions.py:173: RuntimeWarning: The columns ['F_x__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"'
 'F_x__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"mean"'
 'F_x__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"min"' ...
 'T_z__fft_coefficient__attr_"real"__coeff_98'
 'T_z__fft_coefficient__attr_"real"__coeff_99'
 'T_z__spkt_welch_density__coeff_8'] did not have any finite values. Filling with zeros.
  df.iloc[:, np.where(is_col_non_finite)[0]].columns.values), RuntimeWarning)


In [50]:
import pandas as pd

columns = np.array(list(features_filtered.columns.values))

features_names = [None] * int(features_filtered.shape[1]);


for i in range (columns.shape[0]):
    name = columns[i]
    c = '__';
    words = name.split(c)
    
    features_names[i] = 'X__' + str(c.join(words[1:]))


In [56]:
outfile1 = TemporaryFile()

In [57]:
np.save(outfile1, features_names)

In [60]:
_ = outfile1.seek(0)
np.load(outfile1)

array(['X__value_count__value_-1', 'X__abs_energy',
       'X__range_count__max_1__min_-1', 'X__abs_energy',
       'X__standard_deviation', 'X__variance',
       'X__fft_coefficient__attr_"abs"__coeff_1',
       'X__fft_coefficient__attr_"abs"__coeff_1', 'X__abs_energy',
       'X__standard_deviation', 'X__variance',
       'X__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"',
       'X__standard_deviation', 'X__variance',
       'X__ratio_value_number_to_time_series_length',
       'X__standard_deviation', 'X__variance',
       'X__fft_coefficient__attr_"abs"__coeff_1',
       'X__fft_coefficient__attr_"abs"__coeff_2',
       'X__cid_ce__normalize_True', 'X__partial_autocorrelation__lag_1',
       'X__autocorrelation__lag_1',
       'X__percentage_of_reoccurring_values_to_all_values',
       'X__fft_coefficient__attr_"abs"__coeff_4',
       'X__ratio_value_number_to_time_series_length',
       'X__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"',
       'X

In [61]:
def from_columns(columns, columns_to_ignore=None):
    """
    Creates a mapping from kind names to fc_parameters objects
    (which are itself mappings from feature calculators to settings)
    to extract only the features contained in the columns.
    To do so, for every feature name in columns this method

    1. split the column name into col, feature, params part
    2. decide which feature we are dealing with (aggregate with/without params or apply)
    3. add it to the new name_to_function dict
    4. set up the params

    :param columns: containing the feature names
    :type columns: list of str
    :param columns_to_ignore: columns which do not contain tsfresh feature names
    :type columns_to_ignore: list of str

    :return: The kind_to_fc_parameters object ready to be used in the extract_features function.
    :rtype: dict
    """

    kind_to_fc_parameters = {}

    if columns_to_ignore is None:
        columns_to_ignore = []

    for col in columns:
        if col in columns_to_ignore:
            continue

        if not isinstance(col, str):
            raise TypeError("Column name {} should be a string or unicode".format(col))

        # Split according to our separator into <col_name>, <feature_name>, <feature_params>
        parts = col.split('__')
        n_parts = len(parts)

        if n_parts == 1:
            raise ValueError("Splitting of columnname {} resulted in only one part.".format(col))

        kind = parts[0]
        feature_name = parts[1]

        if kind not in kind_to_fc_parameters:
            kind_to_fc_parameters[kind] = {}

        if not hasattr(feature_calculators, feature_name):
            raise ValueError("Unknown feature name {}".format(feature_name))

        config = get_config_from_string(parts)

        if config:
            if feature_name in kind_to_fc_parameters[kind]:
                if config not in kind_to_fc_parameters[kind][feature_name]:
                    kind_to_fc_parameters[kind][feature_name].append(config)
            else:
                kind_to_fc_parameters[kind][feature_name] = [config]
        else:
            kind_to_fc_parameters[kind][feature_name] = None

    return kind_to_fc_parameters

In [62]:
kind_to_fc_parameters = from_columns(features_names)
my_parameters = kind_to_fc_parameters['X']

In [63]:
extract_features(timeseries, column_id="id", column_sort="time", default_fc_parameters=my_parameters)

Feature Extraction: 100%|██████████| 10/10 [00:07<00:00,  1.35it/s]


variable,F_x__abs_energy,F_x__absolute_sum_of_changes,"F_x__agg_autocorrelation__f_agg_""mean""__maxlag_40","F_x__agg_autocorrelation__f_agg_""median""__maxlag_40","F_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","F_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","F_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","F_x__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","F_x__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""max""","F_x__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""mean""",...,T_z__symmetry_looking__r_0.1,T_z__symmetry_looking__r_0.15000000000000002,T_z__symmetry_looking__r_0.25,T_z__time_reversal_asymmetry_statistic__lag_2,T_z__value_count__value_-1,T_z__value_count__value_0,T_z__value_count__value_1,T_z__variance,T_z__variance_larger_than_standard_deviation,T_z__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
1,14.0,2.0,-0.106351,-7.206633e-02,0.0,-0.9,-1.0,0.09,-0.166667,-0.833333,...,0.0,0.0,0.0,0.000000,0.0,15.0,0.0,0.000000,0.0,NaN
2,25.0,14.0,-0.039098,-4.935275e-02,0.0,-0.7,-3.0,0.81,0.000000,-0.566667,...,0.0,0.0,0.0,0.000000,4.0,11.0,0.0,0.195556,0.0,-1.658312
3,12.0,10.0,-0.029815,1.301043e-17,1.0,-0.5,-1.0,0.45,0.500000,-0.566667,...,0.0,0.0,0.0,-0.090909,4.0,11.0,0.0,0.195556,0.0,-1.658312
4,16.0,17.0,-0.049773,-6.417112e-02,1.0,-0.4,-2.0,1.24,1.166667,-0.400000,...,0.0,0.0,1.0,-0.181818,6.0,8.0,1.0,0.355556,0.0,-1.788854
5,17.0,13.0,-0.061467,-5.172414e-02,2.0,-0.5,-2.0,1.05,0.666667,-0.600000,...,1.0,1.0,1.0,-0.090909,4.0,9.0,2.0,0.382222,0.0,-4.636809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,96833.0,100.0,-0.435813,-8.343714e-01,-25.0,-55.3,-110.0,1216.61,-15.500000,-32.933333,...,0.0,0.0,1.0,-3182.363636,1.0,1.0,0.0,93.315556,1.0,-0.624569
85,1683.0,19.0,-0.599870,-4.380362e-01,12.0,6.7,4.0,5.01,6.666667,4.533333,...,0.0,0.0,1.0,-57.545455,1.0,1.0,0.0,4.648889,1.0,-0.621960
86,83497.0,127.0,-0.603352,-4.802260e-01,70.0,40.4,21.0,236.84,27.833333,22.166667,...,0.0,1.0,1.0,141.000000,0.0,0.0,1.0,29.840000,1.0,-1.011593


In [64]:
# Principal Component Analysis
from numpy import array
from sklearn.decomposition import PCA
# define a matrix
A = array([[1, 2], [3, 4], [5, 6]])
print(A)
print('___')
# create the PCA instance
pca = PCA(1)
# fit on data
pca.fit(A)
# access values and vectors
print(pca.components_)
print(pca.explained_variance_)
print('___')
# transform data
B = pca.transform(A)
print(B)
print('___')


[[1 2]
 [3 4]
 [5 6]]
___
[[0.70710678 0.70710678]]
[8.]
___
[[-2.82842712]
 [ 0.        ]
 [ 2.82842712]]
___


In [65]:
outfile2 = TemporaryFile()

In [66]:
np.save(outfile2, pca)

In [67]:
_ = outfile2.seek(0)
np.load(outfile2,allow_pickle=True)

array(PCA(n_components=1), dtype=object)

In [68]:
# add data to the seed data-set
C = array([[1, 2], [3, 4], [5, 6],[7,8],[9,10]])
D = pca.transform(C)
print(D)

[[-2.82842712]
 [ 0.        ]
 [ 2.82842712]
 [ 5.65685425]
 [ 8.48528137]]
